In [1]:
import pandas as pd

##### General notes:
- Solar generation is not normalized, with a max clean value of 0.04, so eps should be 0.0001 or something super small
- I suspect there's a similar issue with net comsumption, since it seems to be the sum of solar generation and non shitable load (and the battery). from the analysis stuck on my rmc machine...
- net_electricity_consumption 0.016, so eps should be ~0.03 of this.
- Price is time based (difference if often 0), so perturbations are obvious, this is not the case for realtime pricing
- Larger changes to SoC can result from attack actions

##### Load data

In [2]:
df_variation = pd.DataFrame({})
df_variation['PPO_baseline'] = pd.read_csv('20 bin PPO 500 results/baseline_obs.csv',
                                            index_col=0,
                                            ).diff().mean()
df_variation['dynamic_uACG'] = pd.read_csv(r'20 bin PPO 500 results\adv_perturbed_obs.csv',
                                            index_col=0,
                                            ).diff().mean()
df_variation['PPO_bifurcated_myPGD_05'] = pd.read_csv(r'20 bin PPO 500 results\untargeted_binary_myPGD_05_adv_obs-a.csv',
                                            index_col=0,
                                            usecols = lambda x: x != 'a', #excludes a col
                                            ).diff().mean()
df_variation['PPO_bifurcated_ACG_05'] = pd.read_csv(r'20 bin PPO 500 results\binary classifier uACG results\uACG DLloss 05 eps adv a-obs.csv',
                                                                index_col=0,
                                                                usecols = lambda x: x != 'a', #excludes a col
                                                                ).diff().mean() 
df_variation['PPO_myPGD_05'] = pd.read_csv(r'20 bin PPO 500 results\untargeted_myPGD_05_adv_obs-a.csv',
                                            index_col=0,
                                            usecols = lambda x: x != 'a', #excludes a col
                                            ).diff().mean()
df_variation['PPO_bifurcated_myPGD_03_mask_time'] = pd.read_csv(r'20 bin PPO 500 results\untargeted_binary_myPGD_03_mask_time_adv_obs-a.csv',
                                                                index_col=0,
                                                                usecols = lambda x: x != 'a', #excludes a col
                                                                ).diff().mean()
df_variation['PPO_bifurcated_myPGD_03_mask_time_time_scale_solar_and_consumption_eps'] = pd.read_csv(
                                                                r'20 bin PPO 500 results\untargeted_binary_myPGD_03_mask_time_scale_solar_and_consumption_eps_adv_obs-a.csv',
                                                                index_col=0,
                                                                usecols = lambda x: x != 'a', #excludes a col
                                                                ).diff().mean()
df_variation['PPO_bifurcated_ACG_03_mask_solar_time_and_consumption_adv_obs'] = pd.read_csv(
                                                                r'20 bin PPO 500 results\binary classifier uACG results\bifurcated_uACG_DLloss_03_mask_time_solar_and_consumption adv a-obs.csv',
                                                                index_col=0,
                                                                usecols = lambda x: x != 'a', #excludes a col
                                                                ).diff().mean()
df_variation['SAC_baseline'] = pd.read_csv(r'default SAC 500 norm space results\baseline_obs-a.csv',
                                            index_col=0,
                                            usecols = lambda x: x != 'a', #excludes a col
                                            ).diff().mean()                                          
df_variation['SAC_bifurcated_myPGD_03_mask_time'] = pd.read_csv(r'default SAC 500 norm space results\untargeted_binary_myPGD_03_mask_time_adv_obs-a.csv',
                                                                index_col=0,
                                                                usecols = lambda x: x != 'a', #excludes a col
                                                                ).diff().mean()
df_variation['SAC_bifurcated_myPGD_03_mask_time_time_scale_solar_and_consumption_eps'] = pd.read_csv(
                                                                r'default SAC 500 norm space results\untargeted_binary_myPGD_03_mask_time_scale_solar_and_consumption_eps_adv_obs-a.csv',
                                                                index_col=0,
                                                                usecols = lambda x: x != 'a', #excludes a col
                                                                ).diff().mean()
df_variation['SAC_bifurcated_myPGD_03_mask_solar_and_time_scale_consumption_eps_clipped_adv_obs'] = pd.read_csv(
                                                                r'default SAC 500 norm space results\untargeted_binary_myPGD_03_mask_solar_and_time_scale_consumption_eps_clipped_adv_obs_adv_obs-a.csv',
                                                                index_col=0,
                                                                usecols = lambda x: x != 'a', #excludes a col
                                                                ).diff().mean()               
df_variation['SAC_bifurcated_ACG_03_mask_solar_time_and_consumption_adv_obs'] = pd.read_csv(
                                                                r'default SAC 500 norm space results\bifurcated_uACG_DLloss_03_mask_time_solar_and_consumption adv a-obs.csv',
                                                                index_col=0,
                                                                usecols = lambda x: x != 'a', #excludes a col
                                                                ).diff().mean()        
df_variation['SAC_toggle_bifurcated_myPGD_03_mask_time'] = pd.read_csv(r'default SAC 500 norm space results\toggle_binary_myPGD_03_mask_time_adv_obs-a.csv',
                                                                index_col=0,
                                                                usecols = lambda x: x != 'a', #excludes a col
                                                                ).diff().mean()      
df_variation['SAC_toggle_bifurcated_myPGD_03_mask_solar_and_time_scale_consumption_eps_clipped_adv_obs'] = pd.read_csv(
                                                                r'default SAC 500 norm space results\toggle_binary_myPGD_03_mask_time_and_solar_scale_consumption_eps_clipped_adv_obs-a.csv',
                                                                index_col=0,
                                                                usecols = lambda x: x != 'a', #excludes a col
                                                                ).diff().mean() 

Load the spread for non prediction or temporal observations

In [3]:
obs_spread = pd.read_csv('citylearn_challenge_2022_phase_2 spread.csv',
                            index_col=0)

##### Analyze Dynamic uACG

Notice how bother SoC's and net consumption's variation are orders of magnitude above the others, perhaps the epsilon should be reduced? Or is the net consumption just affected by the SoC/the attack? The only features with significant variations are related to the agent's actions which the attack is meant to change

In [4]:
abs(df_variation['dynamic_uACG'] - df_variation['PPO_baseline']).sort_values(ascending=False)

electricity_pricing                           4.567253e-06
electricity_pricing_predicted_12h             4.567253e-06
direct_solar_irradiance_predicted_24h         4.567253e-06
outdoor_dry_bulb_temperature_predicted_6h     4.567253e-06
outdoor_dry_bulb_temperature                  4.567253e-06
outdoor_relative_humidity_predicted_12h       4.567249e-06
diffuse_solar_irradiance_predicted_12h        4.567249e-06
outdoor_relative_humidity                     4.567248e-06
direct_solar_irradiance_predicted_12h         4.567246e-06
electrical_storage_soc                        4.538504e-06
month_sin                                     4.501137e-06
non_shiftable_load                            4.359615e-06
outdoor_dry_bulb_temperature_predicted_24h    4.341459e-06
solar_generation                              4.334776e-06
day_type_sin                                  4.201804e-06
diffuse_solar_irradiance_predicted_6h         3.888882e-06
net_electricity_consumption                   2.738522e-

##### Analyze bifurcated myPDG $\epsilon = 0.05$ for the discrete PPO

In [12]:
abs(df_variation['PPO_bifurcated_myPGD_05'] - df_variation['PPO_baseline']).sort_values(ascending=False)

outdoor_relative_humidity_predicted_24h       1.141813e-05
direct_solar_irradiance_predicted_12h         1.127541e-05
month_sin                                     1.127541e-05
direct_solar_irradiance_predicted_24h         1.127541e-05
outdoor_relative_humidity_predicted_12h       1.113268e-05
outdoor_relative_humidity                     1.113268e-05
electricity_pricing                           1.113268e-05
electricity_pricing_predicted_6h              1.113268e-05
day_type_cos                                  1.041905e-05
outdoor_dry_bulb_temperature_predicted_6h     9.848139e-06
carbon_intensity                              9.491322e-06
outdoor_relative_humidity_predicted_6h        9.419955e-06
net_electricity_consumption                   9.418114e-06
diffuse_solar_irradiance_predicted_12h        9.277227e-06
electricity_pricing_predicted_24h             8.849052e-06
electricity_pricing_predicted_12h             8.278146e-06
direct_solar_irradiance                       6.708151e-

##### Analyze myPDG $\epsilon = 0.05$ for the discrete PPO

Deviation in mean feature variation from baseline

In [13]:
abs(df_variation['PPO_myPGD_05'] - df_variation['PPO_baseline']).sort_values(ascending=False)

carbon_intensity                              1.141813e-05
electricity_pricing_predicted_24h             1.141813e-05
outdoor_relative_humidity_predicted_24h       1.141813e-05
electricity_pricing_predicted_6h              1.141813e-05
day_type_sin                                  1.141813e-05
solar_generation                              9.848140e-06
diffuse_solar_irradiance_predicted_6h         9.705412e-06
day_type_cos                                  5.280886e-06
direct_solar_irradiance_predicted_6h          2.997260e-06
electrical_storage_soc                        1.800645e-06
net_electricity_consumption                   1.200148e-07
hour_cos                                      4.567253e-12
outdoor_dry_bulb_temperature_predicted_12h    4.567253e-12
outdoor_dry_bulb_temperature                  3.425440e-12
outdoor_relative_humidity_predicted_6h        3.425440e-12
diffuse_solar_irradiance_predicted_12h        3.425439e-12
direct_solar_irradiance_predicted_12h         3.425439e-

The variation is many orders of magnetude lower for some feautres compared to any other attack, but this is not replication in the absolute variation. Is this because the attack is not bifurcated or masked? Top features variation is an OoM higher than most other attacks too

##### Analyze bifurcated myPDG $\epsilon = 0.03$ with time mask for the discrete PPO

In [14]:
abs(df_variation['PPO_bifurcated_myPGD_03_mask_time'] - df_variation['PPO_baseline']).sort_values(ascending=False)

electricity_pricing                           6.850879e-06
electricity_pricing_predicted_24h             6.850879e-06
electricity_pricing_predicted_6h              6.850879e-06
direct_solar_irradiance                       6.850879e-06
diffuse_solar_irradiance_predicted_24h        6.850879e-06
direct_solar_irradiance_predicted_24h         6.850879e-06
outdoor_relative_humidity_predicted_12h       6.850876e-06
carbon_intensity                              6.850873e-06
outdoor_relative_humidity_predicted_24h       6.850872e-06
diffuse_solar_irradiance_predicted_12h        6.850872e-06
outdoor_relative_humidity                     6.850870e-06
direct_solar_irradiance_predicted_12h         6.850870e-06
outdoor_relative_humidity_predicted_6h        6.850869e-06
net_electricity_consumption                   6.711770e-06
outdoor_dry_bulb_temperature                  6.708153e-06
outdoor_dry_bulb_temperature_predicted_6h     6.708153e-06
diffuse_solar_irradiance                      6.708152e-

##### Analyze bifurcated myPDG $\epsilon = 0.03$ with time mask for the discrete SAC

In [15]:
abs(df_variation['SAC_bifurcated_myPGD_03_mask_time'] - df_variation['SAC_baseline']).sort_values(ascending=False)

outdoor_dry_bulb_temperature_predicted_12h    6.850879e-06
direct_solar_irradiance_predicted_6h          6.850879e-06
outdoor_relative_humidity_predicted_12h       6.850871e-06
outdoor_dry_bulb_temperature_predicted_6h     5.851793e-06
electricity_pricing_predicted_12h             5.566339e-06
diffuse_solar_irradiance                      4.995433e-06
direct_solar_irradiance                       4.709979e-06
electricity_pricing_predicted_6h              3.996346e-06
direct_solar_irradiance_predicted_12h         3.568166e-06
electricity_pricing                           3.568166e-06
diffuse_solar_irradiance_predicted_12h        3.282716e-06
outdoor_dry_bulb_temperature                  3.282713e-06
diffuse_solar_irradiance_predicted_24h        3.282713e-06
direct_solar_irradiance_predicted_24h         3.282713e-06
diffuse_solar_irradiance_predicted_6h         2.997259e-06
outdoor_relative_humidity_predicted_24h       2.569083e-06
outdoor_relative_humidity_predicted_6h        1.569983e-

##### Analyze bifurcated myPGD $\epsilon = 0.03$ with time mask and scaling $\epsilon$ for solar generation and net consumption for the discrete PPO

In [16]:
abs(df_variation['PPO_bifurcated_myPGD_03_mask_time_time_scale_solar_and_consumption_eps'] - df_variation['SAC_baseline']).sort_values(ascending=False)

electricity_pricing                           6.850879e-06
electricity_pricing_predicted_24h             6.850879e-06
electricity_pricing_predicted_6h              6.850879e-06
diffuse_solar_irradiance_predicted_24h        6.850879e-06
direct_solar_irradiance                       6.850879e-06
direct_solar_irradiance_predicted_24h         6.850879e-06
outdoor_relative_humidity_predicted_12h       6.850876e-06
carbon_intensity                              6.850873e-06
outdoor_relative_humidity_predicted_24h       6.850872e-06
diffuse_solar_irradiance_predicted_12h        6.850872e-06
outdoor_relative_humidity                     6.850870e-06
direct_solar_irradiance_predicted_12h         6.850870e-06
outdoor_relative_humidity_predicted_6h        6.850869e-06
electricity_pricing_predicted_12h             6.708152e-06
outdoor_dry_bulb_temperature                  6.422699e-06
outdoor_dry_bulb_temperature_predicted_6h     6.422699e-06
diffuse_solar_irradiance                      6.422699e-

##### Analyze bifurcated myPDG $\epsilon = 0.03$ with time mask and scaling $\epsilon$ for solar generation and net consumption for the SAC

In [19]:
abs(df_variation['SAC_bifurcated_myPGD_03_mask_time_time_scale_solar_and_consumption_eps'] - df_variation['SAC_baseline']).sort_values(ascending=False)

electricity_pricing_predicted_12h             6.850879e-06
outdoor_dry_bulb_temperature_predicted_6h     6.850879e-06
direct_solar_irradiance_predicted_6h          6.850879e-06
outdoor_relative_humidity_predicted_12h       6.850871e-06
diffuse_solar_irradiance_predicted_24h        6.779516e-06
direct_solar_irradiance                       6.708152e-06
outdoor_relative_humidity                     6.422697e-06
outdoor_relative_humidity_predicted_24h       6.422692e-06
outdoor_dry_bulb_temperature_predicted_12h    5.851793e-06
direct_solar_irradiance_predicted_12h         5.280885e-06
outdoor_dry_bulb_temperature_predicted_24h    3.425436e-06
electrical_storage_soc                        3.411562e-06
electricity_pricing_predicted_6h              2.854533e-06
diffuse_solar_irradiance                      2.140900e-06
electricity_pricing                           1.284540e-06
non_shiftable_load                            1.213178e-06
diffuse_solar_irradiance_predicted_12h        9.990900e-

Variation in net consumption might be related to SoC, which is the attacks goal

##### Analyze SAC bifurcated myPDG $\epsilon = 0.03$ with:
- solar generation and time mask
- scaling $\epsilon$ for net consumption, and
- clipped adversarial observations

In [20]:
abs(df_variation['SAC_bifurcated_myPGD_03_mask_solar_and_time_scale_consumption_eps_clipped_adv_obs'] - df_variation['SAC_baseline']).sort_values(ascending=False)

electricity_pricing_predicted_12h             6.850879e-06
outdoor_relative_humidity_predicted_6h        6.850876e-06
outdoor_relative_humidity_predicted_12h       6.850871e-06
outdoor_relative_humidity                     6.850870e-06
outdoor_dry_bulb_temperature_predicted_6h     6.779516e-06
outdoor_dry_bulb_temperature_predicted_24h    6.422699e-06
outdoor_relative_humidity_predicted_24h       6.279966e-06
diffuse_solar_irradiance_predicted_6h         4.709979e-06
direct_solar_irradiance_predicted_12h         4.424525e-06
electricity_pricing                           3.996346e-06
outdoor_dry_bulb_temperature_predicted_12h    3.853624e-06
direct_solar_irradiance                       3.425440e-06
direct_solar_irradiance_predicted_6h          3.425439e-06
diffuse_solar_irradiance_predicted_24h        3.282713e-06
diffuse_solar_irradiance_predicted_12h        1.998170e-06
electrical_storage_soc                        1.992600e-06
non_shiftable_load                            1.569993e-

##### Analyze bifurcated ACG $\epsilon = 0.03$ masking time, solar generation and net consumption for the SAC

In [21]:
abs(df_variation['PPO_bifurcated_ACG_03_mask_solar_time_and_consumption_adv_obs'] - df_variation['SAC_baseline']).sort_values(ascending=False)

electricity_pricing_predicted_24h             0.046665
electricity_pricing                           0.044899
electricity_pricing_predicted_6h              0.036767
electricity_pricing_predicted_12h             0.032924
hour_sin                                      0.031521
diffuse_solar_irradiance_predicted_24h        0.016108
diffuse_solar_irradiance                      0.016089
electrical_storage_soc                        0.014769
diffuse_solar_irradiance_predicted_6h         0.010154
outdoor_dry_bulb_temperature_predicted_12h    0.009364
outdoor_dry_bulb_temperature_predicted_6h     0.008768
direct_solar_irradiance_predicted_24h         0.008707
diffuse_solar_irradiance_predicted_12h        0.005737
carbon_intensity                              0.005183
outdoor_relative_humidity_predicted_6h        0.004909
hour_cos                                      0.004717
direct_solar_irradiance_predicted_12h         0.002965
direct_solar_irradiance                       0.002604
outdoor_dr

Smaller adv regret and variation in net consumption, suggesting the two are related

##### Analyze bifurcated ACG $\epsilon = 0.03$ masking time, solar generation and net consumption for the SAC

In [22]:
abs(df_variation['SAC_bifurcated_ACG_03_mask_solar_time_and_consumption_adv_obs'] - df_variation['SAC_baseline']).sort_values(ascending=False)

non_shiftable_load                            6.850881e-06
day_type_cos                                  6.850875e-06
outdoor_relative_humidity                     5.964300e-06
electricity_pricing_predicted_24h             5.152778e-06
diffuse_solar_irradiance_predicted_6h         4.002420e-06
outdoor_relative_humidity_predicted_12h       3.978123e-06
electricity_pricing_predicted_12h             3.722854e-06
outdoor_relative_humidity_predicted_24h       3.671756e-06
direct_solar_irradiance_predicted_24h         3.425440e-06
outdoor_dry_bulb_temperature                  3.315990e-06
direct_solar_irradiance_predicted_6h          3.268645e-06
direct_solar_irradiance                       3.212179e-06
net_electricity_consumption                   2.921514e-06
carbon_intensity                              2.693236e-06
outdoor_dry_bulb_temperature_predicted_6h     2.565000e-06
electricity_pricing                           1.971250e-06
hour_cos                                      1.945638e-

##### Analyze toggle bifurcated myPGD $\epsilon = 0.03$ masking time

In [23]:
abs(df_variation['SAC_toggle_bifurcated_myPGD_03_mask_time'] - df_variation['SAC_baseline']).sort_values(ascending=False)

direct_solar_irradiance_predicted_6h          6.850880e-06
electricity_pricing_predicted_6h              6.850879e-06
electricity_pricing_predicted_12h             6.850879e-06
electricity_pricing                           6.850879e-06
diffuse_solar_irradiance_predicted_24h        6.850879e-06
direct_solar_irradiance_predicted_24h         6.850879e-06
diffuse_solar_irradiance_predicted_6h         6.850879e-06
outdoor_dry_bulb_temperature_predicted_24h    6.850876e-06
outdoor_relative_humidity_predicted_12h       6.850876e-06
outdoor_relative_humidity_predicted_24h       6.850875e-06
outdoor_relative_humidity                     6.850870e-06
outdoor_relative_humidity_predicted_6h        6.850869e-06
direct_solar_irradiance_predicted_12h         6.850869e-06
outdoor_dry_bulb_temperature_predicted_12h    6.850867e-06
carbon_intensity                              6.779517e-06
net_electricity_consumption                   6.753873e-06
electrical_storage_soc                        6.313511e-

##### Analyze toggle bifurcated myPGD $\epsilon = 0.03$ masking time, solar generation and net consumption for the SAC

This has a lower adv regret than the corresponding bifurcation attack

In [24]:
abs(df_variation['SAC_toggle_bifurcated_myPGD_03_mask_solar_and_time_scale_consumption_eps_clipped_adv_obs'] - df_variation['SAC_baseline']).sort_values(ascending=False)

direct_solar_irradiance_predicted_6h          6.850880e-06
electricity_pricing_predicted_6h              6.850879e-06
electricity_pricing_predicted_24h             6.850879e-06
electricity_pricing                           6.850879e-06
electricity_pricing_predicted_12h             6.850879e-06
outdoor_relative_humidity_predicted_12h       6.850876e-06
outdoor_relative_humidity                     6.850870e-06
outdoor_relative_humidity_predicted_6h        6.850869e-06
outdoor_dry_bulb_temperature_predicted_12h    6.850867e-06
direct_solar_irradiance_predicted_12h         6.708147e-06
diffuse_solar_irradiance_predicted_6h         6.089530e-06
outdoor_dry_bulb_temperature                  5.709066e-06
electrical_storage_soc                        3.643102e-06
diffuse_solar_irradiance_predicted_24h        3.425440e-06
diffuse_solar_irradiance                      3.425440e-06
direct_solar_irradiance                       5.709066e-07
carbon_intensity                              5.709055e-

By multipling select feature variations by their spread, we no have the variations in their original units

In [25]:
(abs(df_variation['SAC_bifurcated_myPGD_03_mask_solar_and_time_scale_consumption_eps_clipped_adv_obs'] - df_variation['SAC_baseline']).mul(obs_spread.T)).T

,spread
carbon_intensity,7.879259e-21
day_type_cos,NaN
day_type_sin,NaN
diffuse_solar_irradiance,1.451530e-04
diffuse_solar_irradiance_predicted_12h,2.032139e-03
diffuse_solar_irradiance_predicted_24h,3.338519e-03
diffuse_solar_irradiance_predicted_6h,4.790049e-03
direct_solar_irradiance,3.264444e-03
direct_solar_irradiance_predicted_12h,4.216572e-03
direct_solar_irradiance_predicted_24h,1.088148e-03


For this attack, the effect on the mean variation is negligable.